<a href="https://colab.research.google.com/github/laisab/IC/blob/main/rascunho_ic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tabulate import tabulate

In [3]:
# Dicionário com datasets e seus respectivos targets
dicio_datasets_targets = {
    'Adult.csv': 'income'
}

**IndianLiverPatientDataset**

V1. Age of the patient. Any patient whose age exceeded 89 is listed as being of age "90".

V2. Gender of the patient

In [4]:
# Dicionário com datasets e seus respectivos atributos sensíveis
dicio_datasets_sensivel = {
    'Adult.csv': ['age', 'education', 'education.num', 'marital.status', 'relationship', 'race', 'sex']
}

In [5]:
caminho_drive = '/content/drive/MyDrive/IC/Datasets/'
novo_caminho = '/content/drive/MyDrive/Teste/'

In [19]:
# Separação de dados
def separar_feature_target(caminho_completo, atr_sensivel):

  try:
    df = pd.read_csv(caminho_completo)

    # Obtém o nome do target
    nome_arquivo = os.path.basename(caminho_completo)
    nome_target = dicio_datasets_targets.get(nome_arquivo)

    if not nome_target:
      raise ValueError(f'Coluna target não encontrada para o arquivo {nome_arquivo}')

    # Separa features (X) e target (y)
    X = df.drop(columns=[nome_target])
    y = df[nome_target]

    atributo_sensivel = dicio_datasets_sensivel.get(nome_arquivo)

    if not atributo_sensivel:
      raise ValueError(f'Atributos sensíveis não encontrados para o arquivo {nome_arquivo}')
    else:
      atr_sensivel_df = df[atributo_sensivel].copy()

    # print(f'DataFrame de atributo sensível: {atr_sensivel_df}')

    # Codificação de features categóricas com OneHotEncoder
    colunas_categoricas = X.select_dtypes(include=['object']).columns

    if colunas_categoricas.size > 0: # Verificação de colunas categóricas para evitar erros com datasets numéricos
      enc = OneHotEncoder(handle_unknown='ignore') # Lida com categorias desconhecidas em dados futuros
      X_encoded = enc.fit_transform(X[colunas_categoricas]).toarray()
      X_encoded_df = pd.DataFrame(X_encoded, columns=enc.get_feature_names_out(colunas_categoricas))
      X = X.select_dtypes(exclude=['object']) # Remove as colunas categóricas originais
      X = pd.concat([X, X_encoded_df], axis=1)

    return X, y, atr_sensivel_df

  except Exception as e:
    print(f'\nErro ao separar os dados de {nome_arquivo}: {e}')
    return None

In [20]:
# Predição dos datasets
def predizer_dataset(modelo, X, y, cv):
  try:
    y_pred = cross_val_predict(modelo, X, y, cv=cv, method='predict')
    y_prob = cross_val_predict(modelo, X, y, cv=cv, method='predict_proba')
    return y_pred, y_prob

  except Exception as e:
    print(f'\nErro ao predizer dataset: {e}')
    return None

In [8]:
metricas = {
  'accuracy': accuracy_score,
  'f1_score': lambda y_true, y_pred: f1_score(y_true, y_pred, average='macro'),
  'roc_auc_score': lambda y_true, y_pred_proba: roc_auc_score(y_true, y_pred_proba, average='macro', multi_class='ovr') if len(np.unique(y_true)) > 2 else roc_auc_score(y_true, y_pred_proba[:, 1])
}

In [21]:
# Avaliação dos datasets
def avaliar_dataset(y, y_pred, y_prob, metricas):
  try:
    avaliacao = {}
    for metrica, metrica_funcao in metricas.items():
      try:
        if metrica == 'roc_auc_score':
          avaliacao[metrica] = metrica_funcao(y, y_prob)
        else:
          avaliacao[metrica] = metrica_funcao(y, y_pred)
      except Exception as e:
        print(f'\nErro ao calcular {metrica}: {e}')
        avaliacao[metrica] = None

    return avaliacao

  except Exception as e:
    print(f'\nErro ao avaliar dataset: {e}')
    return None

In [60]:
# Avaliação dos segmentos de atributos sensíveis
def avaliar_segmento(y, y_pred, y_prob, atr_sensivel, metricas):
  try:
    avaliacao_segmento = {}

      # atr_sensivel.head().iterrows()

    for indice, valor in atr_sensivel.iterrows():
       segmento = valor.to_dict()
       nome_segmento = ", ".join([f"{chave}={valor}" for chave, valor in segmento.items()])

       mascara = (atr_sensivel.index == indice)

       y_segmento = y[mascara]
       y_pred_segmento = y_pred[mascara]
       y_prob_segmento = y_prob[mascara]

       avaliacao_segmento[nome_segmento] = {}

       for metrica, metrica_funcao in metricas.items():
         try:
           if metrica == 'roc_auc_score':
            if len(np.unique(y_segmento)) > 1:
              avaliacao_segmento[nome_segmento][metrica] = metrica_funcao(y_segmento, y_prob_segmento)
            else:
              print(f'Apenas uma classe presente no segmento {nome_segmento}. AUC não calculada.')
           else:
             avaliacao_segmento[nome_segmento][metrica] = metrica_funcao(y_segmento, y_pred_segmento)
         except Exception as e:
           print(f'\nErro ao calcular {metrica} para a segmentação {nome_segmento}: {e}')
           avaliacao_segmento[nome_segmento][metrica] = None

    return avaliacao_segmento

  except Exception as e:
    print(f'\nErro ao avaliar segmento: {e}')
    return None

In [11]:
# Lista os arquivos csv na pasta
arquivos = [f for f in os.listdir(caminho_drive) if f.endswith('.csv') and f in dicio_datasets_targets]
arquivos

['Adult.csv']

In [61]:
# Processamento de cada dataset
resultados = []
res_metrica_tabela = []
res_segmento_tabela = []

modelo = RandomForestClassifier(n_estimators=20, random_state=42)
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for arquivo in arquivos:
  caminho_completo = os.path.join(caminho_drive, arquivo)
  dados = separar_feature_target(caminho_completo, dicio_datasets_sensivel)

  if dados is not None:
    X, y, atr_sensivel_df = dados
    predicao, probabilidade = predizer_dataset(modelo, X, y, cv)
    resultados = avaliar_dataset(y, predicao, probabilidade, metricas) # Avaliação dos datasets
    resultados_segmentados = avaliar_segmento(y, predicao, probabilidade, atr_sensivel_df, metricas) # Avaliação dos segmentos

    print(resultados)

    # Tabela das métricas para os datasets
    res_metrica_tabela.append([
      arquivo,
      modelo,
      resultados.get('accuracy'),
      resultados.get('f1_score'),
      resultados.get('roc_auc_score')
    ])

    res_segmento_tabela.append([
      arquivo,
      resultados_segmentados.get('accuracy'),
      resultados_segmentados.get('f1_score'),
      resultados_segmentados.get('roc_auc_score')
    ])

    arquivo_pred = f"Predicao_{arquivo}"
    caminho_pred = os.path.join(novo_caminho, arquivo_pred)
    pd.DataFrame(predicao, columns=[arquivo]).to_csv(caminho_pred, index=False)

  else:
    print(f'Não foi possível processar o dataset {arquivo}')

A saída de streaming foi truncada nas últimas 5000 linhas.
Apenas uma classe presente no segmento age=40, education=7th-8th, education.num=4, marital.status=Married-civ-spouse, relationship=Husband, race=White, sex=Male. AUC não calculada.
Apenas uma classe presente no segmento age=53, education=Masters, education.num=14, marital.status=Married-civ-spouse, relationship=Husband, race=White, sex=Male. AUC não calculada.
Apenas uma classe presente no segmento age=44, education=Doctorate, education.num=16, marital.status=Never-married, relationship=Not-in-family, race=White, sex=Male. AUC não calculada.
Apenas uma classe presente no segmento age=37, education=5th-6th, education.num=3, marital.status=Never-married, relationship=Other-relative, race=White, sex=Female. AUC não calculada.
Apenas uma classe presente no segmento age=46, education=Bachelors, education.num=13, marital.status=Married-civ-spouse, relationship=Husband, race=White, sex=Male. AUC não calculada.
Apenas uma classe presen

In [55]:
colunas = ['NOME DO DATASET', 'MODELO', 'ACURÁCIA', 'F1', 'AUC']
print(tabulate(res_metrica_tabela, headers=colunas, tablefmt='fancy_grid'))

╒═══════════════════╤══════════════════════════════════════════════════════════╤════════════╤═════════╤══════════╕
│ NOME DO DATASET   │ MODELO                                                   │   ACURÁCIA │      F1 │      AUC │
╞═══════════════════╪══════════════════════════════════════════════════════════╪════════════╪═════════╪══════════╡
│ Adult.csv         │ RandomForestClassifier(n_estimators=20, random_state=42) │   0.849483 │ 0.78065 │ 0.891422 │
╘═══════════════════╧══════════════════════════════════════════════════════════╧════════════╧═════════╧══════════╛


In [62]:
colunas = ['NOME DO DATASET', 'ACURÁCIA', 'F1', 'AUC']
print(tabulate(res_segmento_tabela, headers=colunas, tablefmt='fancy_grid'))

╒═══════════════════╤════════════╤══════╤═══════╕
│ NOME DO DATASET   │ ACURÁCIA   │ F1   │ AUC   │
╞═══════════════════╪════════════╪══════╪═══════╡
│ Adult.csv         │            │      │       │
╘═══════════════════╧════════════╧══════╧═══════╛
